In [1]:
import os
import glob
import cv2
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
from scipy import ndimage

In [46]:
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras.preprocessing import image
from keras.applications.mobilenetv2 import MobileNetV2, preprocess_input
from keras.utils import plot_model

In [41]:
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping
from keras.utils import np_utils

from sklearn.model_selection import train_test_split

In [98]:
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Flatten, Activation
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.optimizers import RMSprop, Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.utils import np_utils
from keras import backend as K

In [42]:
data_path = os.path.join(os.getcwd(), 'dataset')
data_class = os.listdir(data_path)

In [83]:
data_x = []
data_y = []

In [84]:
for cl in data_class:
    index = data_class.index(cl)
    path = os.path.join(data_path, cl, '*.png')
    files = glob.glob(path)
    for fl in files:
        img = image.load_img(fl, target_size=(96, 96))
        x = image.img_to_array(img)
        x = preprocess_input(x)
        
        data_x.append(x)
        data_y.append(index)

In [85]:
#convert to numpy
data_x = np.array(data_x, dtype=np.uint8)
data_y = np.array(data_y, dtype=np.uint8)

In [86]:
#convert to float
data_y = np_utils.to_categorical(data_y, len(np.unique(data_y))) #preprocess class labels

In [87]:
x_train, x_test, y_train, y_test = train_test_split(data_x, data_y, test_size=0.2, random_state=42)

In [88]:
def mobilenetv2_model():
    base_model = MobileNetV2(input_shape=(96, 96, 3), weights=None, include_top=False)
    top_layer = GlobalAveragePooling2D()(base_model.output)
    predictions = Dense(25, activation='softmax')(top_layer)
    model = Model(input=base_model.input, output = predictions)
    
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    model.summary()
    
    return model

In [89]:
model = mobilenetv2_model()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_8 (InputLayer)            (None, 96, 96, 3)    0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 97, 97, 3)    0           input_8[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 48, 48, 32)   864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 48, 48, 32)   128         Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu

/home/titut/.local/lib/python3.6/site-packages/ipykernel_launcher.py:5: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  """


In [90]:
callbacks = EarlyStopping(monitor='val_loss', patience=3, verbose=0)
model.fit(x_train, y_train, batch_size=100, epochs=30, shuffle=True, verbose=2, 
          validation_split=0.2, callbacks=[callbacks])

Train on 2840 samples, validate on 711 samples
Epoch 1/30
 - 145s - loss: 2.1994 - acc: 0.3588 - val_loss: 7.9231 - val_acc: 0.2025
Epoch 2/30
 - 123s - loss: 1.6496 - acc: 0.5067 - val_loss: 6.3588 - val_acc: 0.2321
Epoch 3/30
 - 133s - loss: 1.4379 - acc: 0.5676 - val_loss: 7.8431 - val_acc: 0.2841
Epoch 4/30


KeyboardInterrupt: 

In [ ]:
score = model.evaluate(x_test, y_test)

In [ ]:
score

In [102]:
data_x = []
data_y = []

In [103]:
for cl in data_class:
    index = data_class.index(cl)
    path = os.path.join(data_path, cl, '*.png')
    files = glob.glob(path)
    for fl in files:
        img = cv2.imread(fl)
        img = cv2.resize(img, (64, 64), interpolation=cv2.INTER_CUBIC)
        data_x.append(img)
        data_y.append(index)

In [104]:
#convert to numpy
data_x = np.array(data_x, dtype=np.uint8)
data_y = np.array(data_y, dtype=np.uint8)

In [105]:
data_y = np_utils.to_categorical(data_y, 25) #preprocess class labels

In [106]:
x_train, x_test, y_train, y_test = train_test_split(data_x, data_y, test_size=0.2, random_state=42)

In [107]:
def center_normalize(x):
    return x - K.mean(x)/K.std(x)

In [108]:
def cnn_model():
    model = Sequential()
    
    model.add(Activation(activation=center_normalize, input_shape=(64, 64, 3)))

    model.add(Conv2D(32, (5, 5), padding='same', activation='relu', data_format='channels_last'))
    model.add(Conv2D(32, (5, 5), padding='same', activation='relu', data_format='channels_last'))
    model.add(MaxPooling2D(pool_size=(2, 2), data_format='channels_last'))
    
    model.add(Conv2D(64, (3, 3), padding='same', activation='relu', data_format='channels_last'))
    model.add(Conv2D(64, (3, 3), padding='same', activation='relu', data_format='channels_last'))
    model.add(MaxPooling2D(pool_size=(2, 2), data_format='channels_last'))
    
    model.add(Conv2D(128, (3, 3), padding='same', activation='relu', data_format='channels_last'))
    model.add(Conv2D(128, (3, 3), padding='same', activation='relu', data_format='channels_last'))
    model.add(MaxPooling2D(pool_size=(2, 2), data_format='channels_last'))
    
    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.5))
    
    model.add(Dense(25, activation='softmax'))

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

In [109]:
model = cnn_model()

In [110]:
filepath="weights-{epoch:02d}-{val_acc:.2f}.h5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callback = [checkpoint]

model.fit(x_train, y_train, batch_size=100, epochs=30, shuffle=True, verbose=1,
          validation_split=0.2, callbacks=callback)

model.load_weights(filepath=filepath)
scores = model.evaluate(x_valid, y_valid, batch_size=100)

Train on 2840 samples, validate on 711 samples
Epoch 1/30
2840/2840 [==============================] - 95s 34ms/step - loss: 3.4051 - acc: 0.1405 - val_loss: 2.4093 - val_acc: 0.2785

Epoch 00001: val_acc improved from -inf to 0.27848, saving model to weights-01-0.28.h5
Epoch 2/30
2840/2840 [==============================] - 76s 27ms/step - loss: 2.5211 - acc: 0.2838 - val_loss: 2.0625 - val_acc: 0.4276

Epoch 00002: val_acc improved from 0.27848 to 0.42757, saving model to weights-02-0.43.h5
Epoch 3/30
2840/2840 [==============================] - 77s 27ms/step - loss: 2.0866 - acc: 0.4046 - val_loss: 1.5097 - val_acc: 0.5457

Epoch 00003: val_acc improved from 0.42757 to 0.54571, saving model to weights-03-0.55.h5
Epoch 4/30
2840/2840 [==============================] - 77s 27ms/step - loss: 1.6896 - acc: 0.5130 - val_loss: 1.2469 - val_acc: 0.6653

Epoch 00004: val_acc improved from 0.54571 to 0.66526, saving model to weights-04-0.67.h5
Epoch 5/30
2840/2840 [==========================

OSError: Unable to open file (unable to open file: name = 'weights-{epoch:02d}-{val_acc:.2f}.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [112]:
model.load_weights(filepath="weights-29-0.90.h5")

NameError: name 'x_valid' is not defined

In [113]:
scores = model.evaluate(x_test, y_test, batch_size=100)

888/888 [==============================] - 7s 8ms/step


In [114]:
scores

[0.46899507872693175, 0.9009009097610507]

In [116]:
pred = model.predict_classes(x_test)

In [119]:
y_true = np.argmax(y_test, axis=1)

In [122]:
y_true[:6]

array([ 9, 19, 13,  1,  6, 17])

In [123]:
pred[:6]

array([ 9,  8, 13,  1,  6, 17])

In [115]:
from sklearn.metrics import classification_report

In [125]:
print(classification_report(y_pred=pred, y_true=y_true))

              precision    recall  f1-score   support

           1       0.95      0.92      0.93       119
           2       0.67      0.80      0.73         5
           3       0.00      0.00      0.00         4
           4       0.82      0.94      0.87        85
           5       0.00      0.00      0.00         4
           6       0.96      0.96      0.96       172
           7       0.86      0.90      0.88        63
           8       0.93      0.95      0.94        41
           9       0.75      0.67      0.71        27
          11       1.00      0.33      0.50         3
          12       0.00      0.00      0.00         3
          13       0.95      0.99      0.97       138
          14       0.00      0.00      0.00         1
          16       0.96      0.92      0.94        52
          17       0.86      0.98      0.92        55
          18       0.69      0.92      0.79        36
          19       0.75      0.27      0.40        11
          20       0.92    

In [111]:
filepath

'weights-{epoch:02d}-{val_acc:.2f}.h5'